KNN

In [1]:
import os
import time
import zipfile
import pickle
import numpy as np

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.random_projection import johnson_lindenstrauss_min_dim, SparseRandomProjection
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import make_scorer, precision_score, recall_score, accuracy_score
from sklearn.model_selection import StratifiedKFold, GridSearchCV
from sklearn.pipeline import Pipeline

###############################################################################
# Utility functions

def extract_zip(zip_file, extract_to):
    """Extracts a zip file to the specified directory."""
    with zipfile.ZipFile(zip_file, 'r') as zip_ref:
        zip_ref.extractall(extract_to)

def getDataPoints(path):
    """Collects content of all .py files within the given directory."""
    dataPointsList = []
    if not os.path.exists(path):
        print(f"Directory does not exist: {path}")
        return dataPointsList

    for root, dirs, files in os.walk(path):
        for dataPointName in files:
            if dataPointName.endswith(".py"):  # Only consider Python files
                file_path = os.path.join(root, dataPointName)
                with open(file_path, encoding="utf-8") as fileIn:
                    dp = fileIn.read().strip()
                    if dp:  # Ensure the document is not empty
                        dataPointsList.append(dp)
                    else:
                        print(f"Empty file: {file_path}")
    
    if len(dataPointsList) == 0:
        print(f"No valid documents found in directory: {path}")
    
    return dataPointsList

def flastVectorization(dataPoints, dim=0, eps=0.3):
    countVec = CountVectorizer(stop_words=None)  # No stop word removal
    Z_full = countVec.fit_transform(dataPoints)
    if eps == 0:
        Z = Z_full
    else:
        if dim <= 0:
            dim = johnson_lindenstrauss_min_dim(Z_full.shape[0], eps=eps)
        srp = SparseRandomProjection(n_components=dim)
        Z = srp.fit_transform(Z_full)
    return Z

###############################################################################
# KNN with GridSearchCV and Multiple Scoring Metrics

def flastKNNWithGridSearchCV(outDir, flakyZip, nonFlakyZip, extractDir, n_splits, dim, eps):
    v0 = time.perf_counter()

    # Extract the zip files
    flakyDir = os.path.join(extractDir, 'flaky')
    nonFlakyDir = os.path.join(extractDir, 'nonFlaky')
    os.makedirs(flakyDir, exist_ok=True)
    os.makedirs(nonFlakyDir, exist_ok=True)
    
    extract_zip(flakyZip, flakyDir)
    extract_zip(nonFlakyZip, nonFlakyDir)

    dataPointsFlaky = getDataPoints(flakyDir)
    dataPointsNonFlaky = getDataPoints(nonFlakyDir)
    dataPoints = dataPointsFlaky + dataPointsNonFlaky

    print(f"Number of flaky documents: {len(dataPointsFlaky)}")
    print(f"Number of non-flaky documents: {len(dataPointsNonFlaky)}")
    print(f"Total number of documents: {len(dataPoints)}")
    if len(dataPoints) > 0:
        print(f"Sample document: {dataPoints[0]}")

    if len(dataPoints) == 0:
        raise ValueError("No documents available for vectorization. Please check the input directories.")

    # Vectorization
    Z = flastVectorization(dataPoints, dim=dim, eps=eps)
    dataPointsList = np.array([Z[i].toarray() for i in range(Z.shape[0])])
    dataLabelsList = np.array([1]*len(dataPointsFlaky) + [0]*len(dataPointsNonFlaky))
    vecTime = time.perf_counter() - v0

    # Define the pipeline for KNN
    pipeline = Pipeline([
        ('knn', KNeighborsClassifier())
    ])

    # Define parameter grid for hyperparameter tuning
    param_grid = {
        'knn__n_neighbors': [3, 5, 7],  # Hyperparameter for k in KNN
        'knn__metric': ['cosine', 'euclidean'],  # Distance metrics
    }

    # Custom scoring functions for precision, recall, and accuracy
    scoring = {
        'precision': make_scorer(precision_score),
        'recall': make_scorer(recall_score),
        'accuracy': make_scorer(accuracy_score)
    }

    # Cross-validation
    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

    # Perform GridSearchCV with multiple scoring metrics
    grid_search = GridSearchCV(pipeline, param_grid, cv=skf, scoring=scoring, refit='accuracy', verbose=1, return_train_score=True)

    # Fit the GridSearchCV on training data
    grid_search.fit(dataPointsList.reshape(len(dataPointsList), -1), dataLabelsList)

    # Get the best parameters and the best score for accuracy
    best_params = grid_search.best_params_
    best_score = grid_search.best_score_

    print(f"Best Parameters: {best_params}")
    print(f"Best Accuracy Score: {best_score}")

    # Save the results
    outFile = f"params-knn-{n_splits}-folds.csv"
    with open(os.path.join(outDir, outFile), "w") as fo:
        fo.write("n_neighbors,metric,accuracy,precision,recall,preparationTime\n")
        for idx, param in enumerate(grid_search.cv_results_['params']):
            accuracy = grid_search.cv_results_['mean_test_accuracy'][idx]
            precision = grid_search.cv_results_['mean_test_precision'][idx]
            recall = grid_search.cv_results_['mean_test_recall'][idx]
            preparationTime = vecTime / len(dataPointsList)  # Estimating preparation time
            fo.write(f"{param['knn__n_neighbors']},{param['knn__metric']},{accuracy},{precision},{recall},{preparationTime}\n")

    print(f"KNN analysis completed for {n_splits}-folds. Results saved to: {outFile}")
    return best_params, best_score

if __name__ == "__main__":
    # Parameters setup
    flakyZip = "compressedDataset/cleaned_flaky_files.zip"
    nonFlakyZip = "compressedDataset/reduced_nonflaky_files.zip"
    extractDir = "extracted"
    outDir = "results/result_FlastKNN"
    os.makedirs(outDir, exist_ok=True)
    os.makedirs(extractDir, exist_ok=True)

    # Perform KNN analysis for 5 folds and 3 folds
    dim = 100  # Example value for JL dimensionality reduction
    eps = 0.3  # JL epsilon

    print("Starting 5-fold analysis...")
    best_params_5folds, best_score_5folds = flastKNNWithGridSearchCV(outDir, flakyZip, nonFlakyZip, extractDir, 5, dim, eps)

    print("Starting 3-fold analysis...")
    best_params_3folds, best_score_3folds = flastKNNWithGridSearchCV(outDir, flakyZip, nonFlakyZip, extractDir, 3, dim, eps)

    print("Best results for 5-fold:")
    print(f"Best Parameters: {best_params_5folds}")
    print(f"Best Accuracy Score: {best_score_5folds}")

    print("Best results for 3-fold:")
    print(f"Best Parameters: {best_params_3folds}")
    print(f"Best Accuracy Score: {best_score_3folds}")


Starting 5-fold analysis...
Number of flaky documents: 47
Number of non-flaky documents: 47
Total number of documents: 94
Sample document: # -*- coding: utf-8 -*-

# This code is part of Qiskit.
#
# (C) Copyright IBM 2020.
#
# This code is licensed under the Apache License, Version 2.0. You may
# obtain a copy of this license in the LICENSE.txt file in the root directory
# of this source tree or at http://www.apache.org/licenses/LICENSE-2.0.
#
# Any modifications or derivative works of this code must retain this
# copyright notice, and modified files need to carry a notice indicating
# that they have been altered from the originals.

"""General utility functions for testing."""

from qiskit import QuantumCircuit
from qiskit.providers.ibmq.ibmqbackend import IBMQBackend


def most_busy_backend(provider):
    """Return the most busy backend for the provider given.

    Return the most busy available backend for those that
    have a `pending_jobs` in their `status`. Backends such as
    

SVM

In [2]:
import os
import time
import zipfile
import numpy as np

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.svm import SVC
from sklearn.metrics import make_scorer, precision_score, recall_score, accuracy_score
from sklearn.model_selection import StratifiedKFold, GridSearchCV
from sklearn.pipeline import Pipeline

###############################################################################
# Utility functions

def extract_zip(zip_file, extract_to):
    """Extracts a zip file to the specified directory."""
    with zipfile.ZipFile(zip_file, 'r') as zip_ref:
        zip_ref.extractall(extract_to)

def getDataPoints(path):
    """Collects content of all .py files within the given directory."""
    dataPointsList = []
    if not os.path.exists(path):
        print(f"Directory does not exist: {path}")
        return dataPointsList

    for root, dirs, files in os.walk(path):
        for dataPointName in files:
            if dataPointName.endswith(".py"):  # Only consider Python files
                file_path = os.path.join(root, dataPointName)
                with open(file_path, encoding="utf-8") as fileIn:
                    dp = fileIn.read().strip()
                    if dp:  # Ensure the document is not empty
                        dataPointsList.append(dp)
                    else:
                        print(f"Empty file: {file_path}")
    
    if len(dataPointsList) == 0:
        print(f"No valid documents found in directory: {path}")
    
    return dataPointsList

def flastVectorization(dataPoints):
    """Vectorizes the data points using CountVectorizer."""
    countVec = CountVectorizer(stop_words=None)  # No stop word removal
    Z = countVec.fit_transform(dataPoints)
    return Z

###############################################################################
# SVM with GridSearchCV and Multiple Scoring Metrics

def flastSVMWithGridSearchCV(outDir, flakyZip, nonFlakyZip, extractDir, n_splits):
    v0 = time.perf_counter()

    # Extract the zip files
    flakyDir = os.path.join(extractDir, 'flaky')
    nonFlakyDir = os.path.join(extractDir, 'nonFlaky')
    os.makedirs(flakyDir, exist_ok=True)
    os.makedirs(nonFlakyDir, exist_ok=True)
    
    extract_zip(flakyZip, flakyDir)
    extract_zip(nonFlakyZip, nonFlakyDir)

    dataPointsFlaky = getDataPoints(flakyDir)
    dataPointsNonFlaky = getDataPoints(nonFlakyDir)
    dataPoints = dataPointsFlaky + dataPointsNonFlaky

    print(f"Number of flaky documents: {len(dataPointsFlaky)}")
    print(f"Number of non-flaky documents: {len(dataPointsNonFlaky)}")
    print(f"Total number of documents: {len(dataPoints)}")
    
    if len(dataPoints) == 0:
        raise ValueError("No documents available for vectorization. Please check the input directories.")

    # Vectorization
    Z = flastVectorization(dataPoints)
    dataLabelsList = np.array([1]*len(dataPointsFlaky) + [0]*len(dataPointsNonFlaky))
    vecTime = time.perf_counter() - v0

    # Define the pipeline for SVM
    pipeline = Pipeline([
        ('svm', SVC())
    ])

    # Define parameter grid for hyperparameter tuning
    param_grid = {
        'svm__C': [0.1, 1.0, 10.0],  # Regularization parameter
        'svm__kernel': ['linear', 'rbf', 'poly', 'sigmoid'],  # Kernel types
    }

    # Custom scoring functions for precision, recall, and accuracy
    scoring = {
        'precision': make_scorer(precision_score, zero_division=0),
        'recall': make_scorer(recall_score, zero_division=0),
        'accuracy': make_scorer(accuracy_score)
    }

    # Cross-validation
    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

    # Perform GridSearchCV with multiple scoring metrics
    grid_search = GridSearchCV(pipeline, param_grid, cv=skf, scoring=scoring, refit='accuracy', verbose=1, return_train_score=True)

    # Fit the GridSearchCV on training data
    grid_search.fit(Z, dataLabelsList)

    # Get the best parameters and the best score for accuracy
    best_params = grid_search.best_params_
    best_score = grid_search.best_score_

    print(f"Best Parameters: {best_params}")
    print(f"Best Accuracy Score: {best_score}")

    # Save the results
    outFile = f"params-svm-{n_splits}-folds.csv"
    with open(os.path.join(outDir, outFile), "w") as fo:
        fo.write("C,kernel,accuracy,precision,recall,preparationTime\n")
        for idx, param in enumerate(grid_search.cv_results_['params']):
            accuracy = grid_search.cv_results_['mean_test_accuracy'][idx]
            precision = grid_search.cv_results_['mean_test_precision'][idx]
            recall = grid_search.cv_results_['mean_test_recall'][idx]
            preparationTime = vecTime / len(dataPoints)  # Estimating preparation time
            fo.write(f"{param['svm__C']},{param['svm__kernel']},{accuracy},{precision},{recall},{preparationTime}\n")

    print(f"SVM analysis completed for {n_splits}-folds. Results saved to: {outFile}")
    return best_params, best_score

if __name__ == "__main__":
    # Parameters setup
    flakyZip = "compressedDataset/cleaned_flaky_files.zip"
    nonFlakyZip = "compressedDataset/reduced_nonflaky_files.zip"
    extractDir = "extracted"
    outDir = "results/reuslt_SVM"
    os.makedirs(outDir, exist_ok=True)
    os.makedirs(extractDir, exist_ok=True)

    # Perform SVM analysis for 5 folds and 3 folds
    print("Starting 5-fold analysis...")
    best_params_5folds, best_score_5folds = flastSVMWithGridSearchCV(outDir, flakyZip, nonFlakyZip, extractDir, 5)

    print("Starting 3-fold analysis...")
    best_params_3folds, best_score_3folds = flastSVMWithGridSearchCV(outDir, flakyZip, nonFlakyZip, extractDir, 3)

    print("Best results for 5-fold:")
    print(f"Best Parameters: {best_params_5folds}")
    print(f"Best Accuracy Score: {best_score_5folds}")

    print("Best results for 3-fold:")
    print(f"Best Parameters: {best_params_3folds}")
    print(f"Best Accuracy Score: {best_score_3folds}")


Starting 5-fold analysis...
Number of flaky documents: 47
Number of non-flaky documents: 47
Total number of documents: 94
Fitting 5 folds for each of 12 candidates, totalling 60 fits
Best Parameters: {'svm__C': 10.0, 'svm__kernel': 'rbf'}
Best Accuracy Score: 0.7444444444444445
SVM analysis completed for 5-folds. Results saved to: params-svm-5-folds.csv
Starting 3-fold analysis...
Number of flaky documents: 47
Number of non-flaky documents: 47
Total number of documents: 94
Fitting 3 folds for each of 12 candidates, totalling 36 fits
Best Parameters: {'svm__C': 10.0, 'svm__kernel': 'rbf'}
Best Accuracy Score: 0.7234543010752689
SVM analysis completed for 3-folds. Results saved to: params-svm-3-folds.csv
Best results for 5-fold:
Best Parameters: {'svm__C': 10.0, 'svm__kernel': 'rbf'}
Best Accuracy Score: 0.7444444444444445
Best results for 3-fold:
Best Parameters: {'svm__C': 10.0, 'svm__kernel': 'rbf'}
Best Accuracy Score: 0.7234543010752689


NB

In [5]:
import os
import time
import zipfile
import pickle
import numpy as np

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import make_scorer, precision_score, recall_score, accuracy_score
from sklearn.model_selection import StratifiedKFold, GridSearchCV
from sklearn.pipeline import Pipeline

###############################################################################
# Utility functions

def extract_zip(zip_file, extract_to):
    """Extracts a zip file to the specified directory."""
    with zipfile.ZipFile(zip_file, 'r') as zip_ref:
        zip_ref.extractall(extract_to)

def getDataPoints(path):
    """Collects content of all .py files within the given directory."""
    dataPointsList = []
    if not os.path.exists(path):
        print(f"Directory does not exist: {path}")
        return dataPointsList

    for root, dirs, files in os.walk(path):
        for dataPointName in files:
            if dataPointName.endswith(".py"):  # Only consider Python files
                file_path = os.path.join(root, dataPointName)
                with open(file_path, encoding="utf-8") as fileIn:
                    dp = fileIn.read().strip()
                    if dp:  # Ensure the document is not empty
                        dataPointsList.append(dp)
                    else:
                        print(f"Empty file: {file_path}")
    
    if len(dataPointsList) == 0:
        print(f"No valid documents found in directory: {path}")
    
    return dataPointsList

def flastVectorization(dataPoints):
    """Vectorizes the data points using CountVectorizer without dimensionality reduction."""
    countVec = CountVectorizer(stop_words=None)  # No stop word removal
    Z = countVec.fit_transform(dataPoints)
    return Z

###############################################################################
# Naive Bayes with GridSearchCV and Multiple Scoring Metrics

def flastNBWithGridSearchCV(outDir, flakyZip, nonFlakyZip, extractDir, n_splits):
    v0 = time.perf_counter()

    # Extract the zip files
    flakyDir = os.path.join(extractDir, 'flaky')
    nonFlakyDir = os.path.join(extractDir, 'nonFlaky')
    os.makedirs(flakyDir, exist_ok=True)
    os.makedirs(nonFlakyDir, exist_ok=True)
    
    extract_zip(flakyZip, flakyDir)
    extract_zip(nonFlakyZip, nonFlakyDir)

    dataPointsFlaky = getDataPoints(flakyDir)
    dataPointsNonFlaky = getDataPoints(nonFlakyDir)
    dataPoints = dataPointsFlaky + dataPointsNonFlaky

    print(f"Number of flaky documents: {len(dataPointsFlaky)}")
    print(f"Number of non-flaky documents: {len(dataPointsNonFlaky)}")
    print(f"Total number of documents: {len(dataPoints)}")
    
    if len(dataPoints) == 0:
        raise ValueError("No documents available for vectorization. Please check the input directories.")

    # Vectorization without Random Projection
    Z = flastVectorization(dataPoints)
    dataLabelsList = np.array([1]*len(dataPointsFlaky) + [0]*len(dataPointsNonFlaky))
    vecTime = time.perf_counter() - v0

    # Define the pipeline for Naive Bayes
    pipeline = Pipeline([
        ('nb', MultinomialNB())
    ])

    # Define parameter grid for hyperparameter tuning
    param_grid = {
        'nb__alpha': [0.01, 0.1, 1.0, 10.0],  # Laplace smoothing parameter
    }

    # Custom scoring functions for precision, recall, and accuracy
    scoring = {
        'precision': make_scorer(precision_score),
        'recall': make_scorer(recall_score),
        'accuracy': make_scorer(accuracy_score)
    }

    # Cross-validation
    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

    # Perform GridSearchCV with multiple scoring metrics
    grid_search = GridSearchCV(pipeline, param_grid, cv=skf, scoring=scoring, refit='accuracy', verbose=1, return_train_score=True)

    # Fit the GridSearchCV on training data
    grid_search.fit(Z, dataLabelsList)

    # Get the best parameters and the best score for accuracy
    best_params = grid_search.best_params_
    best_score = grid_search.best_score_

    print(f"Best Parameters: {best_params}")
    print(f"Best Accuracy Score: {best_score}")

    # Save the results
    outFile = f"params-nb-{n_splits}-folds.csv"
    with open(os.path.join(outDir, outFile), "w") as fo:
        fo.write("alpha,accuracy,precision,recall,preparationTime\n")
        for idx, param in enumerate(grid_search.cv_results_['params']):
            accuracy = grid_search.cv_results_['mean_test_accuracy'][idx]
            precision = grid_search.cv_results_['mean_test_precision'][idx]
            recall = grid_search.cv_results_['mean_test_recall'][idx]
            preparationTime = vecTime / len(dataPoints)  # Estimating preparation time
            fo.write(f"{param['nb__alpha']},{accuracy},{precision},{recall},{preparationTime}\n")

    print(f"Naive Bayes analysis completed for {n_splits}-folds. Results saved to: {outFile}")
    return best_params, best_score

if __name__ == "__main__":
    # Parameters setup
    flakyZip = "compressedDataset/cleaned_flaky_files.zip"
    nonFlakyZip = "compressedDataset/reduced_nonflaky_files.zip"
    extractDir = "extracted"
    outDir = "results/results_NB"
    os.makedirs(outDir, exist_ok=True)
    os.makedirs(extractDir, exist_ok=True)

    # Perform Naive Bayes analysis for 5 folds and 3 folds
    print("Starting 5-fold analysis...")
    best_params_5folds, best_score_5folds = flastNBWithGridSearchCV(outDir, flakyZip, nonFlakyZip, extractDir, 5)

    print("Starting 3-fold analysis...")
    best_params_3folds, best_score_3folds = flastNBWithGridSearchCV(outDir, flakyZip, nonFlakyZip, extractDir, 3)

    print("Best results for 5-fold:")
    print(f"Best Parameters: {best_params_5folds}")
    print(f"Best Accuracy Score: {best_score_5folds}")

    print("Best results for 3-fold:")
    print(f"Best Parameters: {best_params_3folds}")
    print(f"Best Accuracy Score: {best_score_3folds}")


Starting 5-fold analysis...
Number of flaky documents: 47
Number of non-flaky documents: 47
Total number of documents: 94
Fitting 5 folds for each of 4 candidates, totalling 20 fits
Best Parameters: {'nb__alpha': 0.01}
Best Accuracy Score: 0.7760233918128655
Naive Bayes analysis completed for 5-folds. Results saved to: params-nb-5-folds.csv
Starting 3-fold analysis...
Number of flaky documents: 47
Number of non-flaky documents: 47
Total number of documents: 94
Fitting 3 folds for each of 4 candidates, totalling 12 fits
Best Parameters: {'nb__alpha': 0.1}
Best Accuracy Score: 0.7345430107526881
Naive Bayes analysis completed for 3-folds. Results saved to: params-nb-3-folds.csv
Best results for 5-fold:
Best Parameters: {'nb__alpha': 0.01}
Best Accuracy Score: 0.7760233918128655
Best results for 3-fold:
Best Parameters: {'nb__alpha': 0.1}
Best Accuracy Score: 0.7345430107526881


XG Boost

In [2]:
import os
import time
import zipfile
import pickle
import numpy as np
import xgboost as xgb

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.random_projection import johnson_lindenstrauss_min_dim, SparseRandomProjection
from sklearn.metrics import precision_score, recall_score, accuracy_score
from sklearn.model_selection import StratifiedKFold

###############################################################################
# Utility functions

def extract_zip(zip_file, extract_to):
    """Extracts a zip file to the specified directory."""
    with zipfile.ZipFile(zip_file, 'r') as zip_ref:
        zip_ref.extractall(extract_to)

def getDataPoints(path):
    """Collects content of all .py files within the given directory."""
    dataPointsList = []
    if not os.path.exists(path):
        print(f"Directory does not exist: {path}")
        return dataPointsList

    for root, dirs, files in os.walk(path):
        for dataPointName in files:
            if dataPointName.endswith(".py"):  # Only consider Python files
                file_path = os.path.join(root, dataPointName)
                with open(file_path, encoding="utf-8") as fileIn:
                    dp = fileIn.read().strip()
                    if dp:  # Ensure the document is not empty
                        dataPointsList.append(dp)
                    else:
                        print(f"Empty file: {file_path}")
    
    if len(dataPointsList) == 0:
        print(f"No valid documents found in directory: {path}")
    
    return dataPointsList

def computeResults(testLabels, predictLabels):
    precision = precision_score(testLabels, predictLabels, zero_division=0)
    recall = recall_score(testLabels, predictLabels, zero_division=0)
    accuracy = accuracy_score(testLabels, predictLabels)
    return precision, recall, accuracy

###############################################################################
# FLAST functions

def flastVectorization(dataPoints, dim=0, eps=0.3):
    countVec = CountVectorizer(stop_words=None)  # No stop word removal
    Z_full = countVec.fit_transform(dataPoints)
    if eps == 0:
        Z = Z_full
    else:
        if dim <= 0:
            dim = johnson_lindenstrauss_min_dim(Z_full.shape[0], eps=eps)
        srp = SparseRandomProjection(n_components=dim)
        Z = srp.fit_transform(Z_full)
    return Z

def flastXGBClassification(trainData, trainLabels, testData, eta, max_depth, params):
    t0 = time.perf_counter()
    xgb_model = xgb.XGBClassifier(eta=eta, max_depth=max_depth, use_label_encoder=False, eval_metric="logloss")
    xgb_model.fit(trainData, trainLabels)
    trainTime = time.perf_counter() - t0

    t0 = time.perf_counter()
    predictLabels = xgb_model.predict(testData)
    testTime = time.perf_counter() - t0

    return trainTime, testTime, predictLabels

###############################################################################
# FLAST XGB analysis

def flastXGB(outDir, flakyZip, nonFlakyZip, extractDir, kf, dim, eps, eta, max_depth, params):
    v0 = time.perf_counter()

    # Extract the zip files
    flakyDir = os.path.join(extractDir, 'flaky')
    nonFlakyDir = os.path.join(extractDir, 'nonFlaky')
    os.makedirs(flakyDir, exist_ok=True)
    os.makedirs(nonFlakyDir, exist_ok=True)
    
    extract_zip(flakyZip, flakyDir)
    extract_zip(nonFlakyZip, nonFlakyDir)

    dataPointsFlaky = getDataPoints(flakyDir)
    dataPointsNonFlaky = getDataPoints(nonFlakyDir)
    dataPoints = dataPointsFlaky + dataPointsNonFlaky

    print(f"Number of flaky documents: {len(dataPointsFlaky)}")
    print(f"Number of non-flaky documents: {len(dataPointsNonFlaky)}")
    print(f"Total number of documents: {len(dataPoints)}")
    if len(dataPoints) > 0:
        print(f"Sample document: {dataPoints[0]}")

    if len(dataPoints) == 0:
        raise ValueError("No documents available for vectorization. Please check the input directories.")

    Z = flastVectorization(dataPoints, dim=dim, eps=eps)
    dataPointsList = np.array([Z[i].toarray() for i in range(Z.shape[0])])
    dataLabelsList = np.array([1]*len(dataPointsFlaky) + [0]*len(dataPointsNonFlaky))
    vecTime = time.perf_counter() - v0

    # Storage calculation
    xgb_data = (dataPointsList, dataLabelsList)
    pickleDumpXGB = os.path.join(outDir, f"flast-xgb-eta{eta}-depth{max_depth}.pickle")
    with open(pickleDumpXGB, "wb") as pickleFile:
        pickle.dump(xgb_data, pickleFile)
    storage = os.path.getsize(pickleDumpXGB)
    os.remove(pickleDumpXGB)

    avgP, avgR, avgA = 0, 0, 0
    avgTPrep, avgTPred = 0, 0
    avgFlakyTrain, avgNonFlakyTrain, avgFlakyTest, avgNonFlakyTest = 0, 0, 0, 0
    successFold, precisionFold = 0, 0

    for trnIdx, tstIdx in kf.split(dataPointsList, dataLabelsList):
        trainData, testData = dataPointsList[trnIdx], dataPointsList[tstIdx]
        trainLabels, testLabels = dataLabelsList[trnIdx], dataLabelsList[tstIdx]
        if sum(trainLabels) == 0 or sum(testLabels) == 0:
            print("Skipping fold with no flaky/non-flaky examples...")
            continue

        successFold += 1
        avgFlakyTrain += sum(trainLabels)
        avgNonFlakyTrain += len(trainLabels) - sum(trainLabels)
        avgFlakyTest += sum(testLabels)
        avgNonFlakyTest += len(testLabels) - sum(testLabels)

        trainData = trainData.reshape((trainData.shape[0], -1))
        testData = testData.reshape((testData.shape[0], -1))

        trainTime, testTime, predictLabels = flastXGBClassification(trainData, trainLabels, testData, eta, max_depth, params)
        preparationTime = (vecTime * len(trainData) / len(dataPoints)) + trainTime
        predictionTime = (vecTime / len(dataPoints)) + (testTime / len(testData))
        precision, recall, accuracy = computeResults(testLabels, predictLabels)

        print(f"Precision: {precision}, Recall: {recall}, Accuracy: {accuracy}")
        if precision != "-":
            precisionFold += 1
            avgP += precision
            avgA += accuracy
        avgR += recall
        avgTPrep += preparationTime
        avgTPred += predictionTime

    if precisionFold > 0:
        avgP /= precisionFold
        avgA /= precisionFold
    else:
        avgP = avgA = "-"
    avgR /= successFold
    avgTPrep /= successFold
    avgTPred /= successFold
    avgFlakyTrain /= successFold
    avgNonFlakyTrain /= successFold
    avgFlakyTest /= successFold
    avgNonFlakyTest /= successFold

    return avgFlakyTrain, avgNonFlakyTrain, avgFlakyTest, avgNonFlakyTest, avgP, avgR, avgA, storage, avgTPrep, avgTPred

if __name__ == "__main__":
    # Parameters setup
    flakyZip = "cleaned_flaky_files.zip"
    nonFlakyZip = "reduced_nonflaky_files.zip"
    extractDir = "extracted"
    outDir = "results/"
    os.makedirs(outDir, exist_ok=True)
    os.makedirs(extractDir, exist_ok=True)

    numSplit = 10
    kf = StratifiedKFold(n_splits=numSplit)

    outFile = "params-xgb.csv"
    with open(os.path.join(outDir, outFile), "w") as fo:
        fo.write("eta,max_depth,dim,eps,avgFlakyTrain,avgNonFlakyTrain,avgFlakyTest,avgNonFlakyTest,precision,recall,accuracy,storage,preparationTime,predictionTime\n")

    # XGB parameters to vary
    eta_values = [0.1, 0.3, 0.5]  # Learning rate
    depth_values = [3, 5, 7]  # Tree depth
    dim_values = [0, 100, 200]  # Dimensionality reduction
    eps_values = [0.1, 0.3, 0.5]  # JL epsilon

    for eta in eta_values:
        for depth in depth_values:
            for dim in dim_values:
                for eps in eps_values:
                    print(f"eta={eta}, depth={depth}, dim={dim}, eps={eps}")
                    results = flastXGB(outDir, flakyZip, nonFlakyZip, extractDir, kf, dim, eps, eta, depth, {})
                    print(f"Results to be written: eta={eta}, depth={depth}, dim={dim}, eps={eps}, {results}")
                    with open(os.path.join(outDir, outFile), "a") as fo:
                        fo.write("{},{},{},{},{},{},{},{},{},{},{},{},{},{}\n".format(eta, depth, dim, eps, *results))

    print("XGBoost analysis completed. Results saved to:", outFile)

eta=0.1, depth=3, dim=0, eps=0.1
Number of flaky documents: 47
Number of non-flaky documents: 47
Total number of documents: 94
Sample document: # Copyright 2021 The NetKet Authors - All rights reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#    http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

import dataclasses
import abc

from flax.core import freeze


@dataclasses.dataclass(frozen=True)
class ModuleFramework(abc.ABC):
    @staticmethod
    @abc.abstractmethod
    def is_loaded() -> bool:
        pass

    @staticmethod
    @abc.abstractm

C:\Users\haha9\anaconda3\Lib\site-packages\xgboost\core.py:158: UserWarning: [13:10:44] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0015a694724fa8361-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


Precision: 0.6666666666666666, Recall: 0.4, Accuracy: 0.6


C:\Users\haha9\anaconda3\Lib\site-packages\xgboost\core.py:158: UserWarning: [13:10:44] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0015a694724fa8361-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


Precision: 0.8, Recall: 0.8, Accuracy: 0.8


...

C:\Users\haha9\anaconda3\Lib\site-packages\xgboost\core.py:158: UserWarning: [13:11:39] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0015a694724fa8361-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
C:\Users\haha9\anaconda3\Lib\site-packages\xgboost\core.py:158: UserWarning: [13:11:39] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0015a694724fa8361-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
C:\Users\haha9\anaconda3\Lib\site-packages\xgboost\core.py:158: UserWarning: [13:11:39] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0015a694724fa8361-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
C:\Users\haha9\anaconda3\Lib\site-packages\

Precision: 0.8, Recall: 1.0, Accuracy: 0.8888888888888888
Precision: 0.75, Recall: 0.75, Accuracy: 0.7777777777777778
Precision: 0.2, Recall: 0.25, Accuracy: 0.2222222222222222
Results to be written: eta=0.5, depth=7, dim=200, eps=0.1, (42.3, 42.3, 4.7, 4.7, 0.71, 0.7, 0.6900000000000001, 151008, 0.23332005999982358, 0.002633570531724009)
eta=0.5, depth=7, dim=200, eps=0.3
Number of flaky documents: 47
Number of non-flaky documents: 47
Total number of documents: 94
Sample document: # Copyright 2021 The NetKet Authors - All rights reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#    http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the Li

C:\Users\haha9\anaconda3\Lib\site-packages\xgboost\core.py:158: UserWarning: [13:11:40] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0015a694724fa8361-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
C:\Users\haha9\anaconda3\Lib\site-packages\xgboost\core.py:158: UserWarning: [13:11:40] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0015a694724fa8361-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
C:\Users\haha9\anaconda3\Lib\site-packages\xgboost\core.py:158: UserWarning: [13:11:40] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0015a694724fa8361-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
C:\Users\haha9\anaconda3\Lib\site-packages\

Precision: 0.6666666666666666, Recall: 1.0, Accuracy: 0.7777777777777778
Precision: 1.0, Recall: 1.0, Accuracy: 1.0
Precision: 0.4, Recall: 0.5, Accuracy: 0.4444444444444444
Results to be written: eta=0.5, depth=7, dim=200, eps=0.3, (42.3, 42.3, 4.7, 4.7, 0.7841666666666667, 0.7500000000000001, 0.7444444444444445, 151008, 0.24037151000375162, 0.002716695037912277)
eta=0.5, depth=7, dim=200, eps=0.5
Number of flaky documents: 47
Number of non-flaky documents: 47
Total number of documents: 94
Sample document: # Copyright 2021 The NetKet Authors - All rights reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#    http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either expres

C:\Users\haha9\anaconda3\Lib\site-packages\xgboost\core.py:158: UserWarning: [13:11:40] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0015a694724fa8361-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
C:\Users\haha9\anaconda3\Lib\site-packages\xgboost\core.py:158: UserWarning: [13:11:40] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0015a694724fa8361-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
C:\Users\haha9\anaconda3\Lib\site-packages\xgboost\core.py:158: UserWarning: [13:11:40] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0015a694724fa8361-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
C:\Users\haha9\anaconda3\Lib\site-packages\